<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Импорт-библиотек" data-toc-modified-id="Импорт-библиотек-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Импорт библиотек</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Избавление-от-пропусков" data-toc-modified-id="Избавление-от-пропусков-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Избавление от пропусков</a></span></li></ul></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследование задачи</a></span><ul class="toc-item"><li><span><a href="#Модель-случайного-леса-без-учёта-дисбаланса" data-toc-modified-id="Модель-случайного-леса-без-учёта-дисбаланса-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Модель случайного леса без учёта дисбаланса</a></span></li></ul></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Импорт библиотек

In [1]:
import pandas as pd  # импорт библиотеки pandas 
import numpy as np   # импорт библиотеки numpy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler    # импорт структуры для стандартизации данных
from sklearn.ensemble import RandomForestClassifier  # импорт модели случайного леса
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

## Подготовка данных

In [2]:
data = pd.read_csv('/datasets/Churn.csv')     # чтение файла
data.info()                         # вывод информации о данных
data.head()                         # вывод первых 5 строчек датафрейма

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


**Вывод**

Датафрейм содержит пропуски в графе срока владения картой. Названия столбцов не соответствуют принятому стилю. В датафрейме содержатся столбцы, которые никак не должны влиять на наш прогноз: индекс строки в данных, id покупателя, фамилия покупателя. Поэтому целесообразно создать новый датафрейм без лишних столбцов.



In [3]:
data.columns = data.columns.str.lower()   # приведение названия колонок к принятому виду
data.head()                               # вывод первых 5 строчек датафрейма  

,rownumber,customerid,surname,creditscore,geography,gender,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [4]:
df = data.drop(['rownumber', 'customerid', 'surname'], axis=1) # создаём датафрейм без лишней инфрмации
df.head()                                                      # вывод первых 5 строчек датафрейма

,creditscore,geography,gender,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


### Избавление от пропусков


In [5]:
df['tenure'].value_counts()  # считаем распределение годов действия банковского договора

1.0     952
2.0     950
8.0     933
3.0     928
5.0     927
7.0     925
4.0     885
9.0     882
6.0     881
10.0    446
0.0     382
Name: tenure, dtype: int64

**Вывод**

Видна подозрительная просадка количества клиентов с договором 10-летней и 0-летней давности. Кроме того, подозрительно, что количество пропущенных значений в столбце 'tenure' идеально скомпенсировало бы просадки в годах договоров. Нужно обязательно связаться с поставщиком данной таблицы. У нас такой возможности нет.

Будем предполагать, что на месте пропуска должна быть либо 10, либо 0. Поскольку мы не можем идентифицировать кому писать 10 лет, а кому 0, то будем заполнять пропуски случайно. Поскольку положение в таблице не как не связано с годом действия договора, то заполнение пропусков с чётным номером в таблице нулём, а нечётных - 10 тождественно случайному заполнению. 

In [6]:
df.loc[(df['tenure'].isna()) & ((                                                                            # если встречаем
    df['tenure'].index % 2) == 0)] = df[(df['tenure'].isna()) & ((df['tenure'].index % 2) == 0)].fillna(0)   # пропуск на чётном
# месте, заполняем пропуск 0

df.loc[(df['tenure'].isna()) & ((                                                                            # если встречаем
    df['tenure'].index % 2) != 0)] = df[(df['tenure'].isna()) & ((df['tenure'].index % 2) != 0)].fillna(10)  # пропуск на 
# нечётном месте, заполняем пропуск 10
df.info()                                                # смотрим, остались ли пропуски 
df['tenure'].value_counts()                              # смотрим распределение по годам действия банковского договора 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   creditscore      10000 non-null  int64  
 1   geography        10000 non-null  object 
 2   gender           10000 non-null  object 
 3   age              10000 non-null  int64  
 4   tenure           10000 non-null  float64
 5   balance          10000 non-null  float64
 6   numofproducts    10000 non-null  int64  
 7   hascrcard        10000 non-null  int64  
 8   isactivemember   10000 non-null  int64  
 9   estimatedsalary  10000 non-null  float64
 10  exited           10000 non-null  int64  
dtypes: float64(3), int64(6), object(2)
memory usage: 859.5+ KB


1.0     952
2.0     950
8.0     933
3.0     928
5.0     927
7.0     925
4.0     885
9.0     882
6.0     881
0.0     869
10.0    868
Name: tenure, dtype: int64

**Вывод**

Нам удалось избавиться от пропусков. Просадки числа клиентов с 10-летними и 0-летними договорами компенсированы.

In [7]:
df = pd.get_dummies(df,drop_first=True)   # прямое кодирование категориальных величин
df.head()                                 # вывод первых 5 строчек датафрейма

,creditscore,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited,geography_Germany,geography_Spain,gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0


## Исследование задачи

### Баланс классов

Посмотрим баланс двух классов: 'клиент ушёл' - 1, 'клиент остался' - 0.

In [8]:
df['exited'].value_counts()/len(df['exited'])  # определяет относительные размеры двух классов

0    0.7963
1    0.2037
Name: exited, dtype: float64

Виден явный дисбаланс 80% против 20%.

### Создание выборок

Проводим разбиение датафрейма на три выборки: обучающую, валидационную и тестовую.

In [9]:
df_train_valid, df_test = train_test_split(df, test_size=0.2, random_state=12345)        # разбиваем выборку на тестовую
# и оставшуюся
df_train, df_valid = train_test_split(df_train_valid, test_size=0.25, random_state=1234) # оставшуюся выборку разбиваем на 
# обучающую и валидационную
print('Размер обучающей выборки: ', df_train.shape[0])         # выводим размер обучающей выборки
print('Размер тестовой выборки: ', df_test.shape[0])           # выводим размер тестовой выборки
print('Размер валидационной выборки: ',df_valid.shape[0])      # выводим размер валидационной выборки

Размер обучающей выборки:  6000
Размер тестовой выборки:  2000
Размер валидационной выборки:  2000


**Вывод**

Выборки разбиты в соотношении 60%:20%:20%, как и полагалось.

### Выделение признаков и целевых признаков

In [10]:
target_train = df_train['exited']                          # целевой признак в обучающей выборке
features_train = df_train.drop(['exited'] , axis=1)        # признаки в обучающей выборке

target_valid = df_valid['exited']                          # целевой признак в валидационной выборке
features_valid = df_valid.drop(['exited'] , axis=1)        # признаки в валидационной выборке

target_test = df_test['exited']                            # целевой признак в тестовой выборке
features_test = df_test.drop(['exited'] , axis=1)          # признаки в тестовой выборке

### Модель случайного леса без учёта дисбаланса

In [11]:
f1_best = 0                                                                 # начальное значение f1

for depth in range(1,15):                                                   # цикл по глубине леса
    model = RandomForestClassifier(random_state=12345, max_depth=depth)     # модель случайного леса
    model.fit(features_train,target_train)                                  # обучение модели на обучающей выборке
    predicted_valid = model.predict(features_valid)                         # построение предсказаний на валидационной выборке

    probabilities_valid = model.predict_proba(features_valid)   # вычисляем вероятности классов для каждой строки валид. выборки
    probabilities_one_valid = probabilities_valid[:, 1]                      # выбираем вероятности класса 1
    f1 = f1_score(target_valid, predicted_valid)                             # вычисление величины F1
    auc_roc = roc_auc_score(target_valid, probabilities_one_valid)           # вычисление величины AUC-ROC
    if f1 > f1_best:                                                   # если вычисленное F1 получилось больше зафикс. значения
        best_model = model                                                   # сохраняем эту модель
        f1_best = f1                                                         # фиксируем F1
        depth_best = depth                                                   # фиксируем глубину
        auc_roc_best = auc_roc                                               # фиксируем величину AUC-ROC

for estimators in range(30,200,10):
    model = RandomForestClassifier(random_state=12345, max_depth=depth_best, n_estimators=estimators) # модель случайного леса
    model.fit(features_train,target_train)                                   # обучение модели на обучающей выборке
    predicted_valid = model.predict(features_valid)                          # построение предсказаний на валидационной выборке

    probabilities_valid = model.predict_proba(features_valid) # вычисляем вероятности классов для каждой строки валид. выборки
    probabilities_one_valid = probabilities_valid[:, 1]                       # выбираем вероятности класса 1
    f1 = f1_score(target_valid, predicted_valid)                              # вычисление величины F1
    auc_roc = roc_auc_score(target_valid, probabilities_one_valid)            # вычисление величины AUC-ROC
    if f1 > f1_best:                                                    # если вычисленное F1 получилось больше зафикс. значения
        best_model = model                                                     # сохраняем эту модель
        f1_best = f1                                                           # фиксируем F1
        estimators_best = estimators                                           # фиксируем количество деревьев
        auc_roc_best = auc_roc                                                 # фиксируем величину AUC-ROC
       
        
print(f'Величина F1: {f1_best}')                                               # вывод величины F1
print(f'Величина AUC-ROC: {auc_roc_best}')                                     # вывод величины AUC-ROC

Величина F1: 0.5559105431309904
Величина AUC-ROC: 0.8395772703970977


**Вывод**

Величина F1 курам на смех. Дисбаланс в данных привёл к тому, что наша модель не дотягивает до заветной величины F1 в 0.59. Соотношение двух классов 1:4 можно точно считать несбалансированным.

Величина AUC-ROC говорит о том, что модель делает предсказания лучше случайной. Видимо, при дисбалансе это не сложно.

## Борьба с дисбалансом

In [12]:
f1_best = 0                                                                                # начальное значение f1

for depth in range(1,15):                                                                  # цикл по глубине леса
    model = RandomForestClassifier(random_state=12345, class_weight='balanced', max_depth=depth) # модель случайного леса
# со взвешиванием классов
    model.fit(features_train,target_train)                                # обучение модели на обучающей выборке
    predicted_valid = model.predict(features_valid)                       # построение предсказаний на валидационной выборке
    

    probabilities_valid = model.predict_proba(features_valid)   # вычисляем вероятности классов для каждой строки валид. выборки
    probabilities_one_valid = probabilities_valid[:, 1]            # выбираем вероятности класса 1
    f1 = f1_score(target_valid, predicted_valid)                   # вычисление величины F1
    auc_roc = roc_auc_score(target_valid, probabilities_one_valid) # вычисление величины AUC-ROC
    if f1 > f1_best:                                               # если вычисленное F1 получилось больше зафикс. значения
        best_model = model                                         # сохраняем эту модель
        f1_best = f1                                               # фиксируем F1
        depth_best = depth                                         # фиксируем глубину
        auc_roc_best = auc_roc                                     # фиксируем величину AUC-ROC

for estimators in range(30,200,10):                                               # цикл по количеству деревьев
    model = RandomForestClassifier(random_state=12345, class_weight='balanced', max_depth=depth_best, n_estimators=estimators) #
# модель случайного леса со взвешиванием классов
    model.fit(features_train,target_train)
    predicted_valid = model.predict(features_valid)
    

    probabilities_valid = model.predict_proba(features_valid)   # вычисляем вероятности классов для каждой строки валид. выборки
    probabilities_one_valid = probabilities_valid[:, 1]              # выбираем вероятности класса 1
    f1 = f1_score(target_valid, predicted_valid)                     # вычисление величины F1
    auc_roc = roc_auc_score(target_valid, probabilities_one_valid)   # вычисление величины AUC-ROC
    if f1 > f1_best:                                                 # если вычисленное F1 получилось больше зафикс. значения
        best_model = model                                           # сохраняем эту модель
        f1_best = f1                                                 # фиксируем F1
        estimators_best = estimators                                 # фиксируем количество деревьев
        auc_roc_best = auc_roc                                       # фиксируем величину AUC-ROC
        probabilities_one_valid_best = probabilities_one_valid       # фиксируем вероятности класса 1

f1_offset_best = 0                                                   # начальное значение f1 при смещении порога
for threshold in np.arange(0.5, 0.503, 0.001):                       # цикл по величинам порога
    predicted_valid = probabilities_one_valid_best > threshold       # корректировка предсказаний с учётом нового порога
    f1_offset = f1_score(target_valid, predicted_valid)              # вычисление величины F1
    print(f'Величина F1: {f1_offset} при пороге {threshold}')
    if f1_offset > f1_offset_best:                                   # если вычисленное F1 получилось больше зафикс. значения
        f1_offset_best = f1_offset                                   # фиксируем F1
        threshold_best = threshold                                   # фиксируем порог
        
print()
print(f'Лучшая величина F1: {f1_offset_best} c глубиной дерева {depth_best}, количеством деревьев {estimators_best} и величиной порога: {threshold_best}')
print(f'Величина AUC-ROC при лучшем показателе F1: {auc_roc_best}')

Величина F1: 0.6128205128205128 при пороге 0.5
Величина F1: 0.61340206185567 при пороге 0.501
Величина F1: 0.6149870801033592 при пороге 0.502
Величина F1: 0.6131953428201811 при пороге 0.503

Лучшая величина F1: 0.6149870801033592 c глубиной дерева 10, количеством деревьев 80 и величиной порога: 0.502
Величина AUC-ROC при лучшем показателе F1: 0.8482479593043888


**Вывод**

В ходе борьбы с дисбалансом использовалось взвешивание классов и изменение порога классификации. Взвешивание классов действительно, даёт серьёзное повышение точности предсказаний, а вот изменение порога классификации в нашем случае, скорее всего, носит гомеопатический характер: прирост 0.002 - это очень мало.

## Тестирование модели

In [13]:
predicted_test = best_model.predict(features_test)                                  # строим предсказания на тестовой выборке

probabilities_test = best_model.predict_proba(features_test)                        # вычисляем вероятности классов для каждой
# строки тестовой выборки
probabilities_one_test = probabilities_test[:, 1]                                   # выбираем вероятности класса 1

auc_roc = roc_auc_score(target_test, probabilities_one_test)                        # вычисление величины AUC-ROC 


predicted_test = probabilities_one_test > threshold_best                            # смещение порога на оптимальную величину
f1_offset = f1_score(target_test, predicted_test)                                   # вычисление величины F1
        
print(f'Величина F1 на тестовой выборке: {f1_offset}')                              # вывод величины F1
print(f'Величина AUC-ROC на тестовой выборке: {auc_roc}')                           # вывод величины AUC-ROC

Величина F1 на тестовой выборке: 0.6302021403091558
Величина AUC-ROC на тестовой выборке: 0.8646405755198601


**Вывод**

Наша модель случайного леса с оптимальным числом деревьев, глубиной, порогом классификации и взвешиванием классов показала себя хорошо на тесте. Заветное число 0.59 преодолено, величина AUC-ROC также сигнализирует о том, что мы предсказываем лучше случайной модели. 

## Общий вывод

Для построения оптимальной модели предсказаний нам потребовалось подготовить данные: убрать строки никак не влияющие на результат предсказания, заполнить пропуски. Пропуски заполнялись не медианным и не средним. Были обнаружены просадки в количестве клиентов с 0-летними и 10-летними договорами. Величина просадки хорошо совпадает с количеством пропущенных значений в столбце с годами действия договоров. Было принято решение заполнить эти пропуски или 0, или 10 случайным образом (по логике в 50% случаях мы должны угадывать).

Были определены относительные размеры двух интересующих нас классов. Их размеры соотносятся как 1:4.

Далее данные были разбиты на выборки соответствующих размеров, выделены признаки и целевые признаки.

С наскока была принята попытка построить модель случайного леса без учёта дисбаланса класса, но результат на валидационной выборке нас не удовлетворил. Даже использование оптимальной глубины и количества деревьев не приводило нас к заветное величине F1 в 0.59. Были предприняты попытки решения дисбаланса: построение моделей со взвешиванием и смещение порога. Эффект дало взвешивание: прирост F1 на несколько %.

Экзамен на тестовой выборке прошёл даже лучше, чем на валидационной
- величина F1: 0.6302021403091558
- величина AUC-ROC: 0.8646405755198601

Что же касается величины AUC-ROC, то было замечено, что рост F1 ведёт к росту и этой метрики. На всех этапах построения лучшей модели наша величина AUC-ROC была больше 0.5. Видимо, при дисбалансе классов случайная модель ведёт себя не лучшим образом и превзойти её не сложно.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: задача исследована
    - [x]  Исследован баланс классов
    - [x]  Изучены модели без учёта дисбаланса
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 3: учтён дисбаланс
    - [x]  Применено несколько способов борьбы с дисбалансом
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 4: проведено тестирование
- [x]  Удалось достичь *F1*-меры не менее 0.59
- [x]  Исследована метрика *AUC-ROC*